# Preliminaries

In [1]:
%pip install langchain-core langchain-community langchain-openai beautifulsoup4 markdownify python-dotenv ragstack-ai-knowledge-store ragstack-ai-langchain

  Using cached ragstack_ai_langchain-1.1.1-py3-none-any.whl.metadata (1.7 kB)
  Using cached astrapy-1.2.1-py3-none-any.whl.metadata (13 kB)
  Using cached langchain-0.2.3-py3-none-any.whl.metadata (6.9 kB)
  Using cached langchain_astradb-0.3.3-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_community-0.2.4-py3-none-any.whl.metadata (2.4 kB)
  Using cached langchain_core-0.2.5-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain_openai-0.1.8-py3-none-any.whl.metadata (2.5 kB)
  Using cached unstructured-0.14.5-py3-none-any.whl.metadata (28 kB)
  Using cached langsmith-0.1.77-py3-none-any.whl.metadata (13 kB)
  Using cached bson-0.5.10.tar.gz (10 kB)
  Preparing metadata (setup.py) ... done
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached uuid6-2024.1.12-py3-none-any.whl.metadata (8.6 kB)
Using cached langchain_core-0.2.5-py3-none-any.whl (314 kB)
Using cached langchain_community-0.2.4-py3-none-any.whl (2.2 MB)
Using cached langchain-0

In [2]:
%load_ext autoreload
%autoreload 2

# Load the Astra Documentation into Graph Store

First, we'll crawl the DataStax documentation. LangChain includes a `SiteMapLoader` but it loads all of the pages into memory simultaneously, which makes it impossible to index larger sites from small environments (such as CoLab). So, we'll scrape the sitemap ourselves and iterate over the URLs, allowing us to process documents in batches and flush them to Astra DB. 

## Scrape the URLs from the Site Maps
First, we use Beautiful Soup to parse the XML content of each sitemap and get the list of URLs.
We also add a few extra URLs for external sites that are also useful to include in the index.

In [3]:
# Use sitemaps to crawl the content
SITEMAPS = [
    "https://docs.datastax.com/en/sitemap-astra-db-vector.xml",
    "https://docs.datastax.com/en/sitemap-cql.xml",
    "https://docs.datastax.com/en/sitemap-dev-app-drivers.xml",
    "https://docs.datastax.com/en/sitemap-glossary.xml",
    "https://docs.datastax.com/en/sitemap-astra-db-serverless.xml"
]

# Additional URLs to crawl for content.
EXTRA_URLS = [
    "https://github.com/jbellis/jvector"
]

SITE_PREFIX = "astra"

from bs4 import BeautifulSoup
import requests

def load_pages(sitemap_url):
    r = requests.get(sitemap_url,
                     headers={
                         # Astra docs only return a sitemap with a user agent set.
                         "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0",
                     })
    xml = r.text

    soup = BeautifulSoup(xml, features="xml")
    url_tags = soup.find_all("url")
    for url in url_tags:
        yield(url.find("loc").text)


# For maintenance purposes, we could check only the new articles since a given time.
URLS = [
    url
    for sitemap_url in SITEMAPS
    for url in load_pages(sitemap_url)
] + EXTRA_URLS
len(URLS)

1373

## Load the content from each URL
Next, we create the code to load each page. This performs the following steps:

1. Parses the HTML with BeautifulSoup
2. Locates the "content" of the HTML using an appropriate selector based on the URL
3. Find the link (`<a href="...">`) tags in the content and collect the absolute URLs (for creating edges).

Adding the URLs of these references to the metadata allows the graph store to create edges between the document.

In [4]:
from langchain_community.document_loaders import AsyncHtmlLoader
from bs4 import BeautifulSoup
from langchain_core.documents import Document
from typing import AsyncIterator, Iterable
from ragstack_knowledge_store.graph_store import CONTENT_ID
from markdownify import MarkdownConverter
from ragstack_graph_store.extractors import HtmlLinkEdgeExtractor

markdown_converter = MarkdownConverter(heading_style="ATX")
html_link_extractor = HtmlLinkEdgeExtractor()

def select_content(soup: BeautifulSoup, url: str) -> BeautifulSoup:
    if url.startswith("https://docs.datastax.com/en/"):
        return soup.select_one("article.doc")
    elif url.startswith("https://github.com"):
        return soup.select_one("article.entry-content")
    else:
        return soup

async def load_pages(urls: Iterable[str]) -> AsyncIterator[Document]:
    loader = AsyncHtmlLoader(
        urls,
        requests_per_second=4,
        # Astra docs require a user agent
        header_template = {
            "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0"
        }
    )
    async for html in loader.alazy_load():
        url = html.metadata["source"]

        # Use the URL as the content ID.
        html.metadata[CONTENT_ID] = url

        # Apply the selectors while loading. This reduces the size of
        # the document as early as possible for reduced memory usage.
        soup = BeautifulSoup(html.page_content, "html.parser")
        content = select_content(soup, url)

        # Extract HTML links from the content.
        html_link_extractor.extract_one(html, content)

        # Convert the content to markdown
        html.page_content = markdown_converter.convert_soup(content)

        yield html

ImportError: cannot import name 'HtmlLinkEdgeExtractor' from 'ragstack_knowledge_store.langchain.extractors' (unknown location)

## Initialize Environment
Before we initialize the Graph Store and write the documents we need to set some environment variables.
In colab, this will prompt you for input. When running locally, this will load from `.env`.

In [ ]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
    # (Option 1) - Set the environment variables from getpass.
    print("In colab. Using getpass/input for environment variables.")
    import getpass
    import os

    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key: ")
    os.environ["ASTRA_DB_DATABASE_ID"] = input("Enter Astra DB Database ID: ")
    os.environ["ASTRA_DB_APPLICATION_TOKEN"] = getpass.getpass("Enter Astra DB Application Token: ")

    keyspace = input("Enter Astra DB Keyspace (Empty for default): ")
    if keyspace:
        os.environ["ASTRA_DB_KEYSPACE"] = keyspace
    else:
        os.environ.pop("ASTRA_DB_KEYSPACE", None)
else:
    print("Not in colab. Loading '.env' (see 'env.template' for example)")
    import dotenv
    dotenv.load_dotenv()

## Initialize Cassio and Graph Store
With the environment variables set we initialize the Cassio library for talking to Cassandra / Astra DB.
We also create the `GraphStore`.

In [ ]:
SITE_PREFIX="astra_docs"

In [ ]:
answer = input("Drop Tables? [(Y)es/(N)o]")
if answer.lower() in ["y","yes"]:
    import cassio
    cassio.init(auto=True)
    from cassio.config import check_resolve_session, check_resolve_keyspace
    session = check_resolve_session()
    keyspace = check_resolve_keyspace()
    session.execute(f"DROP TABLE IF EXISTS {keyspace}.{SITE_PREFIX}_nodes")
    session.execute(f"DROP TABLE IF EXISTS {keyspace}.{SITE_PREFIX}_targets")
else:
    # Handle no / "wrong" input
    pass

In [ ]:
import cassio
from langchain_openai import OpenAIEmbeddings
from ragstack_langchain.graph_store import CassandraGraphStore

cassio.init(auto=True)
embeddings = OpenAIEmbeddings()
graph_store = CassandraGraphStore(
    embeddings,
    node_table=f"{SITE_PREFIX}_nodes",
    targets_table=f"{SITE_PREFIX}_targets")

## Load the Documents
Finally, we fetch pages and write them to the graph store in batches of 50.

In [ ]:
not_found = 0
found = 0

docs = []
async for doc in load_pages(URLS):
    if doc.page_content.startswith("\n# Page Not Found"):
        not_found += 1
        continue

    docs.append(doc)
    found += 1

    if len(docs) >= 50:
        graph_store.add_documents(docs)
        docs.clear()

if docs:
    graph_store.add_documents(docs)
print(f"{not_found} (of {not_found + found}) URLs were not found")

# Create and execute the RAG Chains

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o")

template = """You are a helpful technical support bot. You should provide complete answers explaining the options the user has available to address their problem. Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    formatted = "\n\n".join(f"From {doc.metadata['content_id']}: {doc.page_content}" for doc in docs)
    return formatted

We'll use the following question. This is an interesting question because the ideal answer should be concise and in-depth, based on how the vector indexing is actually implemented.

In [ ]:
QUESTION="What vector indexing algorithms does Astra use?"

In [ ]:
from IPython.display import display, Markdown

# Helper method to render markdown in responses to a chain.
def run_and_render(chain, question):
    result = chain.invoke(question)
    display(Markdown(result))

## Vector-Only Retrieval

In [ ]:
# Depth 0 doesn't traverses edges and is equivalent to vector similarity only.
vector_retriever = graph_store.as_retriever(search_kwargs={"depth": 0})

vector_rag_chain = (
    {"context": vector_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
run_and_render(vector_rag_chain, QUESTION)

## Graph Traversal Retrieval

In [ ]:
# Depth 1 does vector similarity and then traverses 1 level of edges.
graph_retriever = graph_store.as_retriever(search_kwargs={"depth": 1})

graph_rag_chain = (
    {"context": graph_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
run_and_render(graph_rag_chain, QUESTION)

## MMR Graph Traversal

In [ ]:
mmr_graph_retriever = graph_store.as_retriever(
    search_type = "mmr_traversal",
    search_kwargs = {
        "k": 4,
        "fetch_k": 10,
        "depth": 2,
        # "score_threshold": 0.2,
    },
)

mmr_graph_rag_chain = (
    {"context": mmr_graph_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
run_and_render(mmr_graph_rag_chain, QUESTION)

## Check Retrieval Results

In [ ]:
# Set the question and see what documents each technique retrieves.
for i, doc in enumerate(vector_retriever.invoke(QUESTION)):
  print(f"Vector [{i}]:    {doc.metadata['content_id']}")

for i, doc in enumerate(graph_retriever.invoke(QUESTION)):
  print(f"Graph [{i}]:     {doc.metadata['content_id']}")

for i, doc in enumerate(mmr_graph_retriever.invoke(QUESTION)):
  print(f"MMR Graph [{i}]: {doc.metadata['content_id']}")

# Conclusion
With vector only we retrieved chunks from the Astra documentation explaining that it used JVector.
Since it didn't follow the link to [JVector on GitHub](https://github.com/jbellis/jvector) it didn't actually answer the question.

The graph retrieval started with the same set of chunks, but it followed the edge to the documents we loaded from GitHub.
This allowed the LLM to read in more depth how JVector is implemented, which allowed it to answer the question more clearly and with more detail.